# **Applied Data Science - Capstone Project**
## Best Shop Location in Singapore

- Explore and cluster the neighborhoods in Singapore
- Extract data of districts/areas of Singapore through scrapping from internet 
- Perform analysis based on the data for the information in the district/areas
- Provide maps to visualize neighborhoods and their clustering for insights of the district/areas
- Conclude the best shop location to open in Singapore

In [1]:
# !pip install folium
# !conda install -c conda-forge folium=0.5.0 --yes # uncomment this line if you haven't completed the Foursquare API lab
!conda install -c conda-forge geopy --yes # uncomment this line if you haven't completed the Foursquare API lab

import numpy as np # library to handle data in a vectorized manner
import pandas as pd # library for data analsysis
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import json # library to handle JSON files

from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

import folium # map rendering library

print('Libraries imported.')

Solving environment: done


==> WARNING: A newer version of conda exists. <==
  current version: 4.5.11
  latest version: 4.7.10

Please update conda by running

    $ conda update -n base -c defaults conda



## Package Plan ##

  environment location: /home/jupyterlab/conda/envs/python

  added / updated specs: 
    - geopy


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    geopy-1.20.0               |             py_0          57 KB  conda-forge
    geographiclib-1.49         |             py_0          32 KB  conda-forge
    ------------------------------------------------------------
                                           Total:          90 KB

The following NEW packages will be INSTALLED:

    geographiclib: 1.49-py_0   conda-forge
    geopy:         1.20.0-py_0 conda-forge


geopy-1.20.0         | 57 KB     | ##################################### | 100% 
geographiclib-1.49   | 

In [2]:
!pip install bs4
import requests
from urllib.request import urlopen
from bs4 import BeautifulSoup
import ssl
import csv

     |████████████████████████████████| 102kB 703kB/s ta 0:00:01
  Stored in directory: /home/jupyterlab/.cache/pip/wheels/a0/b0/b2/4f80b9456b87abedbc0bf2d52235414c3467d8889be38dd472
Successfully built bs4


In [11]:
# SSL certification
# ctx = ssl.create_default_context()
# ctx.check_hostname = False
# ctx.verify_mode = ssl.CERT_NONE

src = requests.get('https://en.wikipedia.org/wiki/Postal_codes_in_Singapore').text
soup = BeautifulSoup(src, 'html5lib')

In [12]:
wikiTable = soup.find('table',{'class':'wikitable'})
wikiTable_rows = wikiTable.find_all('tr')

In [13]:
dataExtracted = []
for row in wikiTable_rows:
    dataExtracted.append([t.text.strip() for t in row.find_all('td')])

In [14]:
dfNeigh = pd.DataFrame(dataExtracted, columns=['Postal district', 'Postal sector', 'General location'])
dfNeigh = dfNeigh[~dfNeigh['Postal district'].isnull()]  # to filter out bad rows
dfNeigh.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 28 entries, 1 to 28
Data columns (total 3 columns):
Postal district     28 non-null object
Postal sector       28 non-null object
General location    28 non-null object
dtypes: object(3)
memory usage: 896.0+ bytes


In [15]:
# dfNeigh.drop(dfNeigh[dfNeigh['Borough']=="Not assigned"].index,axis=0, inplace=True)
# dfAssigned = dfNeigh.reset_index()
# dfAssigned.head(10)

In [16]:
dfNeigh.rename(columns={ 'Postal district':'Postal Code',
                         'Postal sector':'Borough',
                         'General location':'Neighbourhood'}, inplace=True)
dfNeigh.head()

,Postal Code,Borough,Neighbourhood
1,01,"01, 02, 03, 04, 05, 06","Raffles Place, Cecil, Marina, People's Park"
2,02,"07, 08","Anson, Tanjong Pagar"
3,03,"14, 15, 16","Bukit Merah, Queenstown, Tiong Bahru"
4,04,"09, 10","Telok Blangah, Harbourfront"
5,05,"11, 12, 13","Pasir Panjang, Hong Leong Garden, Clementi New..."


In [17]:
# dfCombined = dfNeigh.groupby('Postal Code').agg(lambda x: ','.join(x))
# dfCombined.head(10)

In [18]:
# dfCombined.loc[dfCombined['Neighbourhood']=="Not assigned",'Neighbourhood'] = dfCombined.loc[dfCombined['Neighbourhood']=="Not assigned",'Borough']
# dfCombined.head(10)

In [19]:
# Reset the index
# dfFinal = dfCombined.reset_index()
# Remove the duplicate boroughs
# dfFinal['Borough']= dfFinal['Borough'].str.replace('nan|[{}\s]','').str.split(',').apply(set).str.join(',').str.strip(',').str.replace(",{2,}",",")
# dfFinal.head(10)

In [21]:
dfNeigh.shape

(28, 3)

In [27]:
dfNeigh

,Postal Code,Borough,Neighbourhood
1,01,"01, 02, 03, 04, 05, 06","Raffles Place, Cecil, Marina, People's Park"
2,02,"07, 08","Anson, Tanjong Pagar"
3,03,"14, 15, 16","Bukit Merah, Queenstown, Tiong Bahru"
4,04,"09, 10","Telok Blangah, Harbourfront"
5,05,"11, 12, 13","Pasir Panjang, Hong Leong Garden, Clementi New..."
6,06,17,"High Street, Beach Road (part)"
7,07,"18, 19","Middle Road, Golden Mile"
8,08,"20, 21","Little India, Farrer Park, Jalan Besar, Lavender"
9,09,"22, 23","Orchard, Cairnhill, River Valley"
10,10,"24, 25, 26, 27","Ardmore, Bukit Timah, Holland Road, Tanglin"


In [28]:
dfNeigh.to_csv('singaporePostalCode.csv', index=False)

In [29]:
# dfCombined.rename_axis("Postal Code", axis='index', inplace=True)
# dfCombined.head()

In [30]:
# dfCombined.to_csv('singaporePostalCode.csv', index=False)

In [31]:
# Get the location from the csv file
# SgpCoor = 'https://cocl.us/Geospatial_data'
# !wget -q -O 'singaporeCoor.csv' SgpCoor
dfSingapore = pd.read_csv('singaporeCoor.csv').set_index("Postal Code")
dfSingapore.head(10)

,Borough,Neighbourhood,Latitude,Longitude
Postal Code,,,,
1,"01, 02, 03, 04, 05, 06","Raffles Place, Cecil, Marina, People's Park",1.285495,103.850328
2,"07, 08","Anson, Tanjong Pagar",1.276742,103.846673
3,"14, 15, 16","Bukit Merah, Queenstown, Tiong Bahru",1.274279,103.815424
4,"09, 10","Telok Blangah, Harbourfront",1.275670,103.805306
5,"11, 12, 13","Pasir Panjang, Hong Leong Garden, Clementi New...",1.313676,103.766229
6,17,"High Street, Beach Road (part)",1.302047,103.861548
7,"18, 19","Middle Road, Golden Mile",1.299749,103.858188
8,"20, 21","Little India, Farrer Park, Jalan Besar, Lavender",1.304957,103.854093
9,"22, 23","Orchard, Cairnhill, River Valley",1.300759,103.831382


In [32]:
# dfSingapore = pd.read_csv("SingaporeWithCoor.csv").set_index("Borough")
# dfSingapore = pd.read_csv(r"SingaporeWithCoor.csv")
# dfSingapore.head()

In [33]:
print('The dataframe has {} boroughs and {} neighborhoods.'.format(
        len(dfSingapore['Borough'].unique()),
        dfSingapore.shape[0]
    )
)

The dataframe has 28 boroughs and 28 neighborhoods.


In [34]:
# Get the Singapore latitude and longitude
address = 'Singapore, Singapore'

geolocator = Nominatim(user_agent="ny_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Singapore are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Singapore are 1.3408528, 103.878446863736.


In [35]:
# create map of Singapore using latitude and longitude values
map_Singapore = folium.Map(location=[latitude, longitude], zoom_start=11)

# add markers to map
for lat, lng, borough, dfSingapore in zip(dfSingapore['Latitude'], dfSingapore['Longitude'], dfSingapore['Borough'], dfSingapore['Neighbourhood']):
    label = '{}, {}'.format(dfSingapore, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_Singapore)  
    
map_Singapore

- For this task, I will just reduce the our target analysis to Neighbohoods in East,West and Central Singapore only. Lets just take portion of dataframe where Boroughs contain word Singapore.

In [36]:
# dfSingapore = pd.read_csv("SingaporeWithCoor.csv").set_index("Borough")
# dfSingapore = pd.read_csv(r"SingaporeWithCoor.csv")
# dfSingapore.head()

In [37]:
# dfSGP = dfSingapore[dfSingapore['Neighbourhood'].str.contains('Seletar')].reset_index(drop=True)
# dfSGP.head()

In [40]:
# Get the Singapore latitude and longitude
address = 'Singapore, Singapore'

geolocaSGP = Nominatim(user_agent="ny_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Singapore are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Singapore are 1.3408528, 103.878446863736.


In [41]:
# create map of Singapore using latitude and longitude values
map_Singapore = folium.Map(location=[latitude, longitude], zoom_start=11)

# add markers to map
for lat, lng, borough, dfSingapore in zip(dfSingapore['Latitude'], dfSingapore['Longitude'], dfSingapore['Borough'], dfSingapore['Neighbourhood']):
    label = '{}, {}'.format(dfSingapore, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_Singapore)  
    
map_Singapore

TypeError: string indices must be integers

### Define Foursquare Credentials and Version

In [44]:
CLIENT_ID = 'AD1LGMPNUFZ1S1ZQEWBQZFB1RDJ4XGWOGPIHZLJAZFHH31OP' # your Foursquare ID
CLIENT_SECRET = 'OGGOSAT5W1G5WOGKPNRLQJD4S5JUPRUV0WK24MALHZ2AKBLR' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: AD1LGMPNUFZ1S1ZQEWBQZFB1RDJ4XGWOGPIHZLJAZFHH31OP
CLIENT_SECRET:OGGOSAT5W1G5WOGKPNRLQJD4S5JUPRUV0WK24MALHZ2AKBLR


### Let's explore the first neighborhood in our dataframe

In [45]:
# Get the neighbourhood's name
dfSingapore.loc[0, 'Neighbourhood']

AttributeError: 'str' object has no attribute 'loc'

In [49]:
# Get the neighborhood's latitude and longitude values.
neighbourhood_latitude = dfSingapore.loc[0, 'Latitude'] # neighborhood latitude value
neighbourhood_longitude = dfSingapore.loc[0, 'Longitude'] # neighborhood longitude value

neighbourhood_name = dfSingapore.loc[0, 'Neighbourhood'] # neighborhood name

print('Latitude and longitude values of {} are {}, {}.'.format(neighbourhood_name, 
                                                               neighbourhood_latitude, 
                                                               neighbourhood_longitude))

AttributeError: 'str' object has no attribute 'loc'

### Now, let's get the top 100 venues that are in the Beaches within a radius of 500 meters.

In [50]:
LIMIT = 100  # limit of number of venues returned by Foursquare API
radius = 500 # define radius

# create URL
url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
    CLIENT_ID, 
    CLIENT_SECRET, 
    VERSION, 
    neighbourhood_latitude, 
    neighbourhood_longitude, 
    radius, 
    LIMIT)
url # display URL

NameError: name 'neighbourhood_latitude' is not defined

In [51]:
# Send the GET request and examine the results
results = requests.get(url).json()
results

NameError: name 'url' is not defined

### All the information is in the items key. Let's borrow the get_category_type function from the Foursquare lab.

In [52]:
# function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

### Now we are ready to clean the json and structure it into a pandas dataframe.

In [53]:
venues = results['response']['groups'][0]['items']
    
nearby_venues = json_normalize(venues) # flatten JSON

# filter columns
filtered_columns = ['venue.name', 'venue.categories', 'venue.location.lat', 'venue.location.lng']
nearby_venues =nearby_venues.loc[:, filtered_columns]

# filter the category for each row
nearby_venues['venue.categories'] = nearby_venues.apply(get_category_type, axis=1)

# clean columns
nearby_venues.columns = [col.split(".")[-1] for col in nearby_venues.columns]

nearby_venues.head()

NameError: name 'results' is not defined

In [54]:
print('{} venues were returned by Foursquare.'.format(nearby_venues.shape[0]))

NameError: name 'nearby_venues' is not defined

# 2. Explore Neighbourhoods in Singapore
### Let's create a function to repeat the same process to all the neighbourhoods in Singapore

In [55]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighbourhood', 
                  'Neighbourhood Latitude', 
                  'Neighbourhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

#### Now write the code to run the above function on each neighbourhood and create a new dataframe called *SingaporeVenues*.

In [56]:
SingaporeVenues = getNearbyVenues(names=dfSGP['Neighbourhood'],
                                   latitudes=dfSGP['Latitude'],
                                   longitudes=dfSGP['Longitude']
                                  )

NameError: name 'dfSGP' is not defined

### Let's check the size of the resulting dataframe

In [57]:
print(SingaporeVenues.shape)
SingaporeVenues.head()

NameError: name 'SingaporeVenues' is not defined

In [58]:
SingaporeVenues.groupby('Neighbourhood').count()

NameError: name 'SingaporeVenues' is not defined

#### Let's find out how many unique categories can be curated from all the returned venues

In [59]:
print('There are {} uniques categories.'.format(len(SingaporeVenues['Venue Category'].unique())))

NameError: name 'SingaporeVenues' is not defined

## 3. Analyze Each Neighborhood

In [60]:
# one hot encoding
Singapore_onehot = pd.get_dummies(SingaporeVenues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
Singapore_onehot['Neighbourhood'] = SingaporeVenues['Neighbourhood'] 

# move neighborhood column to the first column
fixed_columns = [Singapore_onehot.columns[-1]] + list(Singapore_onehot.columns[:-1])
Singapore_onehot = Singapore_onehot[fixed_columns]

Singapore_onehot.head()

NameError: name 'SingaporeVenues' is not defined

### And let's examine the new dataframe size.

In [61]:
Singapore_onehot.shape

NameError: name 'Singapore_onehot' is not defined

#### Next, let's group rows by neighborhood and by taking the mean of the frequency of occurrence of each category

In [62]:
Singapore_grouped = Singapore_onehot.groupby('Neighbourhood').mean().reset_index()
Singapore_grouped

NameError: name 'Singapore_onehot' is not defined

In [32]:
# Let's confirm the new size
Singapore_grouped.shape

(38, 237)

#### Let's print each neighborhood along with the top 5 most common venues

In [33]:
num_top_venues = 5

for hood in Singapore_grouped['Neighbourhood']:
    print("----"+hood+"----")
    temp = Singapore_grouped[Singapore_grouped['Neighbourhood'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----Adelaide,King,Richmond----
             venue  freq
0      Coffee Shop  0.07
1             Café  0.05
2              Bar  0.04
3       Steakhouse  0.04
4  Thai Restaurant  0.04


----Berczy Park----
                venue  freq
0         Coffee Shop  0.11
1        Cocktail Bar  0.05
2              Bakery  0.05
3  Seafood Restaurant  0.04
4      Farmers Market  0.04


----Brockton,Exhibition Place,Parkdale Village----
            venue  freq
0     Coffee Shop  0.09
1  Breakfast Spot  0.09
2            Café  0.09
3             Gym  0.05
4         Stadium  0.05


----Business Reply Mail Processing Centre 969 Eastern----
                venue  freq
0          Smoke Shop  0.07
1          Restaurant  0.07
2  Light Rail Station  0.07
3              Garden  0.07
4             Brewery  0.07


----CN Tower,Bathurst Quay,Island airport,Harbourfront West,King and Spadina,Railway Lands,South Niagara----
                venue  freq
0     Airport Service  0.19
1      Airport Lounge  0.12
2    Airp

#### Let's put that into a *pandas* dataframe

In [34]:
# First, let's write a function to sort the venues in descending order.
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

#### Now let's create the new dataframe and display the top 10 venues for each neighborhood.

In [35]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighbourhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighbourhoods_venues_sorted = pd.DataFrame(columns=columns)
neighbourhoods_venues_sorted['Neighbourhood'] = Singapore_grouped['Neighbourhood']

for ind in np.arange(Singapore_grouped.shape[0]):
    neighbourhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(Singapore_grouped.iloc[ind, :], num_top_venues)

neighbourhoods_venues_sorted.head()

,Neighbourhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,"Adelaide,King,Richmond",Coffee Shop,Café,Steakhouse,Thai Restaurant,Bar,American Restaurant,Gym,Breakfast Spot,Asian Restaurant,Burger Joint
1,Berczy Park,Coffee Shop,Cocktail Bar,Bakery,Steakhouse,Cheese Shop,Farmers Market,Café,Seafood Restaurant,Beer Bar,Restaurant
2,"Brockton,Exhibition Place,Parkdale Village",Breakfast Spot,Café,Coffee Shop,Climbing Gym,Falafel Restaurant,Convenience Store,Burrito Place,Stadium,Caribbean Restaurant,Bar
3,Business Reply Mail Processing Centre 969 Eastern,Pizza Place,Auto Workshop,Garden Center,Garden,Light Rail Station,Fast Food Restaurant,Farmers Market,Park,Comic Shop,Restaurant
4,"CN Tower,Bathurst Quay,Island airport,Harbourf...",Airport Service,Airport Lounge,Airport Terminal,Boutique,Coffee Shop,Airport,Airport Food Court,Airport Gate,Harbor / Marina,Bar
